In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
from scipy.spatial.distance import mahalanobis as mah_dist # u, v, cov_mat
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))




DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["n_cl"] = len(node_to_connected_clusters[i])
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
# ramps = [0.02, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
def po_linear_model_sqrt_expo(graph, alpha=1, beta=1, sigma=0.1, gamma=1):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * np.sqrt(sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i])

In [ ]:
# unit-level
GATE_hat_list = np.zeros((repeat_num, len(ramps)))

for seed in tqdm(range(repeat_num)):
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_nodes,))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")
        torch.manual_seed(2)
        p_list = ramps[num_step:]                
        g_feat_list = []
        y_list = []
        
        for p in p_list:    
            z = (rollout_index < np.quantile(rollout_index, p))
            
            nx.set_node_attributes(g, 0, "z")
            nx.set_node_attributes(g, {unit:1 for unit in range(num_nodes) if z[unit]}, "z")            
                        
            po_linear_model_sqrt_expo(g, gamma = 1)            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)        
    
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/SQRT_expo_CR_unit.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.02, Loss: 7.7042
Epoch 0, Treat Prop: 0.05, Loss: 2.2370
Epoch 0, Treat Prop: 0.10, Loss: 1.4337
Epoch 0, Treat Prop: 0.25, Loss: 2.3707
Epoch 0, Treat Prop: 0.50, Loss: 3.8195
Epoch 50, Treat Prop: 0.02, Loss: 0.1799
Epoch 50, Treat Prop: 0.05, Loss: 0.1767
Epoch 50, Treat Prop: 0.10, Loss: 0.1098
Epoch 50, Treat Prop: 0.25, Loss: 0.2021
Epoch 50, Treat Prop: 0.50, Loss: 0.4267
Epoch 100, Treat Prop: 0.02, Loss: 0.0711
Epoch 100, Treat Prop: 0.05, Loss: 0.0378
Epoch 100, Treat Prop: 0.10, Loss: 0.0420
Epoch 100, Treat Prop: 0.25, Loss: 0.0809
Epoch 100, Treat Prop: 0.50, Loss: 0.0600
Epoch 150, Treat Prop: 0.02, Loss: 0.0648
Epoch 150, Treat Prop: 0.05, Loss: 0.0335
Epoch 150, Treat Prop: 0.10, Loss: 0.0432
Epoch 150, Treat Prop: 0.25, Loss: 0.0776
Epoch 150, Treat Prop: 0.50, Loss: 0.0614
Epoch 200, Treat Prop: 0.02, Loss: 0.0629
Epoch 200, Treat Prop: 0.05, Loss: 0.0329
Epoch 200, Treat Prop: 0.10, Loss: 0.0429
Epoch 200, Treat Prop: 0.25, Loss: 0.0763
Epoch 2

  0%|          | 1/1000 [00:33<9:25:11, 33.95s/it]

...

0.03230353817343712
Seed: 139
Epoch 0, Treat Prop: 0.02, Loss: 7.6863
Epoch 0, Treat Prop: 0.05, Loss: 2.2425
Epoch 0, Treat Prop: 0.10, Loss: 1.4442
Epoch 0, Treat Prop: 0.25, Loss: 2.3471
Epoch 0, Treat Prop: 0.50, Loss: 3.7678
Epoch 50, Treat Prop: 0.02, Loss: 0.1727
Epoch 50, Treat Prop: 0.05, Loss: 0.1659
Epoch 50, Treat Prop: 0.10, Loss: 0.0968
Epoch 50, Treat Prop: 0.25, Loss: 0.1902
Epoch 50, Treat Prop: 0.50, Loss: 0.3752
Epoch 100, Treat Prop: 0.02, Loss: 0.0691
Epoch 100, Treat Prop: 0.05, Loss: 0.0377
Epoch 100, Treat Prop: 0.10, Loss: 0.0420
Epoch 100, Treat Prop: 0.25, Loss: 0.0754
Epoch 100, Treat Prop: 0.50, Loss: 0.0608
Epoch 150, Treat Prop: 0.02, Loss: 0.0643
Epoch 150, Treat Prop: 0.05, Loss: 0.0329
Epoch 150, Treat Prop: 0.10, Loss: 0.0431
Epoch 150, Treat Prop: 0.25, Loss: 0.0715
Epoch 150, Treat Prop: 0.50, Loss: 0.0604
Epoch 200, Treat Prop: 0.02, Loss: 0.0616
Epoch 200, Treat Prop: 0.05, Loss: 0.0321
Epoch 200, Treat Prop: 0.10, Loss: 0.0424
Epoch 200, Treat Pr

 14%|█▍        | 140/1000 [1:16:20<7:47:59, 32.65s/it]

Seed: 140
Epoch 0, Treat Prop: 0.02, Loss: 7.7095
Epoch 0, Treat Prop: 0.05, Loss: 2.2472
Epoch 0, Treat Prop: 0.10, Loss: 1.4345
Epoch 0, Treat Prop: 0.25, Loss: 2.3561
Epoch 0, Treat Prop: 0.50, Loss: 3.7782
Epoch 50, Treat Prop: 0.02, Loss: 0.2019
Epoch 50, Treat Prop: 0.05, Loss: 0.1947
Epoch 50, Treat Prop: 0.10, Loss: 0.1099
Epoch 50, Treat Prop: 0.25, Loss: 0.2331
Epoch 50, Treat Prop: 0.50, Loss: 0.4980
Epoch 100, Treat Prop: 0.02, Loss: 0.0699
Epoch 100, Treat Prop: 0.05, Loss: 0.0384
Epoch 100, Treat Prop: 0.10, Loss: 0.0422
Epoch 100, Treat Prop: 0.25, Loss: 0.0816
Epoch 100, Treat Prop: 0.50, Loss: 0.0604
Epoch 150, Treat Prop: 0.02, Loss: 0.0629
Epoch 150, Treat Prop: 0.05, Loss: 0.0331
Epoch 150, Treat Prop: 0.10, Loss: 0.0435
Epoch 150, Treat Prop: 0.25, Loss: 0.0775
Epoch 150, Treat Prop: 0.50, Loss: 0.0616
Epoch 200, Treat Prop: 0.02, Loss: 0.0605
Epoch 200, Treat Prop: 0.05, Loss: 0.0323
Epoch 200, Treat Prop: 0.10, Loss: 0.0431
Epoch 200, Treat Prop: 0.25, Loss: 0.07

 14%|█▍        | 141/1000 [1:16:53<7:46:33, 32.59s/it]

Seed: 141
Epoch 0, Treat Prop: 0.02, Loss: 7.7068
Epoch 0, Treat Prop: 0.05, Loss: 2.2376
Epoch 0, Treat Prop: 0.10, Loss: 1.4469
Epoch 0, Treat Prop: 0.25, Loss: 2.3681
Epoch 0, Treat Prop: 0.50, Loss: 3.8165
Epoch 50, Treat Prop: 0.02, Loss: 0.1773
Epoch 50, Treat Prop: 0.05, Loss: 0.1738
Epoch 50, Treat Prop: 0.10, Loss: 0.1093
Epoch 50, Treat Prop: 0.25, Loss: 0.2019
Epoch 50, Treat Prop: 0.50, Loss: 0.4209
Epoch 100, Treat Prop: 0.02, Loss: 0.0755
Epoch 100, Treat Prop: 0.05, Loss: 0.0374
Epoch 100, Treat Prop: 0.10, Loss: 0.0453
Epoch 100, Treat Prop: 0.25, Loss: 0.0798
Epoch 100, Treat Prop: 0.50, Loss: 0.0575
Epoch 150, Treat Prop: 0.02, Loss: 0.0685
Epoch 150, Treat Prop: 0.05, Loss: 0.0334
Epoch 150, Treat Prop: 0.10, Loss: 0.0462
Epoch 150, Treat Prop: 0.25, Loss: 0.0764
Epoch 150, Treat Prop: 0.50, Loss: 0.0591
Epoch 200, Treat Prop: 0.02, Loss: 0.0663
Epoch 200, Treat Prop: 0.05, Loss: 0.0328
Epoch 200, Treat Prop: 0.10, Loss: 0.0457
Epoch 200, Treat Prop: 0.25, Loss: 0.07